In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "../../models/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Llama patching release 2024.8
   \\   /|    GPU: NVIDIA A10G. Max memory: 21.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

../../models/Meta-Llama-3.1-8B-Instruct does not have a padding token! Will use pad_token = <|reserved_special_token_247|>.


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [4]:
template = """<|start_header_id|>user<|end_header_id|>Given an email text, please classify it into one of two categories: Save Email or Phishing Email. A Save Email is a legitimate email that does not pose any security risks or attempt to deceive the recipient. On the other hand, a Phishing Email is an email that attempts to trick the recipient into revealing sensitive information or performing malicious actions.
Analyze the content of the email: Look for any signs of deception or attempts to manipulate the recipient. Phishing emails often contain urgent requests, grammatical errors, or suspicious links. Save Emails, on the other hand, typically have clear and concise content without any malicious intent.

Based on these criteria, please classify the email as either Save Email or Phishing Email.

Just write Save Email or Phishing Email, no reasoning or additional text.

Email:

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

EOS_TOKEN = "<|eot_id|>" # Must add EOS_TOKEN
def formatting_prompts_func(examples,train):
    inputs       = examples["Email Text"]
    outputs      = examples["Email Type"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        if train:
            text = template.format(input)+f"{output}<|eot_id|>"
        else:
            text = template.format(input)
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("csv",data_files="Phishing_Email.csv", split = "train")
dataset=dataset.train_test_split(test_size=0.2)
dataset['train'] = dataset['train'].map(formatting_prompts_func, batched = True,fn_kwargs={"train":True})
dataset['test'] = dataset['test'].map(formatting_prompts_func, batched = True,fn_kwargs={"train":False})

dataset.save_to_disk('.email_dataset')

Map:   0%|          | 0/14920 [00:00<?, ? examples/s]

Map:   0%|          | 0/3730 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14920 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3730 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        report_to="none",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 200,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/14920 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [8]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A10G. Max memory = 21.988 GB.
6.062 GB of memory reserved.


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 14,920 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 200
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.406600
2,2.867200
3,2.601000
4,2.448700
5,2.506300
6,2.253600
7,1.964700
8,1.808500
9,1.623000
10,1.893500


In [10]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

375.4351 seconds used for training.
6.26 minutes used for training.
Peak reserved memory = 9.395 GB.
Peak reserved memory for training = 3.333 GB.
Peak reserved memory % of max memory = 42.728 %.
Peak reserved memory for training % of max memory = 15.158 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [11]:
dataset['test'][0]

{'Unnamed: 0': 13008,
 'Email Text': "ethink about it : 08 / 21 / 01 trips and tips - - enron ' s global travel program tracy ramsey , global strategic sourcing director , will be hosting an espeak session tuesday , august 21 at 10 a . m . houston time to answer all your questions about enron ' s travel program and how you can participate . can ' t make the live event ? no worries . go to espeak ( http : / / ethink . enron . com / espeak / exec / default . asp ) now and submit your question . tracy will try to answer it during her event and you can read the transcript later . you ' ve been using the new feature in the idea vault ! if you didn ' t already know , you can now make a suggestion for implementing or improving on an idea . go to the thinkbank now and share your opinion with other enron innovators .",
 'Email Type': 'Safe Email',
 'text': "<|start_header_id|>user<|end_header_id|>Given an email text, please classify it into one of two categories: Save Email or Phishing Email. A

In [13]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    template.format(
        dataset['test'][0]['Email Text'], # instruction
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<|begin_of_text|><|start_header_id|>user<|end_header_id|>Given an email text, please classify it into one of two categories: Save Email or Phishing Email. A Save Email is a legitimate email that does not pose any security risks or attempt to deceive the recipient. On the other hand, a Phishing Email is an email that attempts to trick the recipient into revealing sensitive information or performing malicious actions.\nAnalyze the content of the email: Look for any signs of deception or attempts to manipulate the recipient. Phishing emails often contain urgent requests, grammatical errors, or suspicious links. Save Emails, on the other hand, typically have clear and concise content without any malicious intent.\n\nBased on these criteria, please classify the email as either Save Email or Phishing Email.\n\nJust write Save Email or Phishing Email, no reasoning or additional text.\n\nEmail:\n\nethink about it : 08 / 21 / 01 trips and tips - - enron's global travel program tracy ramsey, g

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [9]:
# Merge to 16bit
if True: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 131.41 out of 186.68 RAM for saving.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:05<00:00,  5.35it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


In [2]:
from datasets import load_from_disk
dataset = load_from_disk(".email_dataset")
texts = dataset['test']['text']
product_targets = dataset['test']['Email Type']

In [4]:
%%capture
from vllm import LLM, SamplingParams

output_merged_dir = "model"
#output_merged_dir = "model"
llm = LLM(output_merged_dir, download_dir=None, dtype="float16",gpu_memory_utilization=0.9,max_model_len=2000,tensor_parallel_size=4)
print('Done')

In [5]:
sampling_params = SamplingParams(temperature=0, max_tokens=5)

outputs= llm.generate(prompts=texts,sampling_params=sampling_params)
answers = [answer.outputs[0].text.strip() for answer in outputs]

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3730/3730 [09:59<00:00,  6.23it/s, est. speed input: 3992.67 toks/s, output: 20.48 toks/s]


In [9]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

# Assuming result_df is your DataFrame and it's already defined
# Calculate metrics
accuracy = accuracy_score(product_targets, answers)
precision, recall, f1, _ = precision_recall_fscore_support(product_targets, answers, average='weighted')
# Create a DataFrame to display metrics
metrics_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score'],
    'Value': [accuracy, precision, recall, f1]
})

metrics_df.T

/home/ubuntu/envs/api/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ubuntu/envs/api/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,0,1,2,3
Metric,Accuracy,Precision,Recall,F1-score
Value,0.296783,0.342965,0.296783,0.318207


| Metric   | Accuracy | Precision | Recall   | F1-score |
|----------|----------|-----------|----------|----------|
| Value    | 0.296783 | 0.342965  | 0.296783 | 0.318207 |


Fine Tuned

In [6]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

# Assuming result_df is your DataFrame and it's already defined
# Calculate metrics
accuracy = accuracy_score(product_targets, answers)
precision, recall, f1, _ = precision_recall_fscore_support(product_targets, answers, average='weighted')
# Create a DataFrame to display metrics
metrics_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-score'],
    'Value': [accuracy, precision, recall, f1]
})

print(metrics_df.T)

               0          1         2         3
Metric  Accuracy  Precision    Recall  F1-score
Value   0.934853    0.97022  0.934853  0.952016


/home/ubuntu/envs/api/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


| Metric   | Accuracy | Precision | Recall   | F1-score |
|----------|----------|-----------|----------|----------|
| Value    | 0.934853 | 0.97022   | 0.934853 | 0.952016 |
